![alt text](https://github.com/callysto/callysto-sample-notebooks/blob/master/notebooks/images/Callysto_Notebook-Banner_Top_06.06.18.jpg?raw=true)

# Working With Open Data Part 4:  Meteorite Landings and Falls Part 1

In this third installment of our working with open data tutorial series we will be working with another interesting data set which contains the year and location of all recorded meteorite falls until the year 2013. This is an open data set, and is hosted at [this github repository](https://github.com/fleiser/Meteorite-landings/blob/master/Meteorite_Landings.csv). This repository also contains an interesting analysis of this data set as well with many more colorful figures, and interested readers are invited to explore that analysis as well. However, we note that the analysis on that page was _not_ meant to be a tutorial, and it may be difficult to follow for those unfamiliar with Python. 


However, as this is the third installment in the tutorial series, let's continue with our traditional first step and import the required libraries. You'll notice this time we're using a few more than we have in the past. Not to worry however, as for the most part their just extra tools to make some visualizations a little more exciting. We have entered comments as to what each library is for, and we will explain what we're using them for when they come up in the tutorial. 

In [ ]:
# These first three libraries are nothing new!
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Geopandas is very similar to pandas, however it contains extra functionality to work with 
# geospacial data such as latitude and longitude
import geopandas as gpd
# This imports the "Point" function, or a function that creates a point on a geospacial graph.
# this function simply makes things easier to work with in Pandas
from shapely.geometry import Point

# The following three libraries are included in order to make interactive widgets in this notebook.
# More on this later. 
from IPython import display
from ipywidgets import interact_manual
import ipywidgets as widgets



Don't worry too much about those new libraries, as they will be considerably less intimidating once you see how they're similar to what we've already used!

## Gathering The Data

We now are ready to download the data directly from the Github site in the cell below. However, you'll notice that we're using a few more steps in the cell below. These are simply to make handling the data a little more straight forward, and their functionality is commented in the cell below for your convenience. 

In [ ]:
'''
These first two lines should not be surprising anymore! This is how we've been bringing data into
our Jupyter notebooks throughout this tutorial series. 
'''
url = 'https://github.com/fleiser/Meteorite-landings/raw/master/Meteorite_Landings.csv'
landings = pd.read_csv(url)

'''
Here things are new and exciting. What the `pd.to_datetime' function does is convert some text that looks 
like a date into a 'datetime' object inside of pandas. This is convenient for parsing later as we will be able
to search by year, month, and day. We're also using errors = 'coerce', which tells python to ignore any data
points that cannot be coerced into a datetime object. You'll also note that we're redefining our pandas column
'year' in place
'''
landings['year'] = pd.to_datetime(landings['year'], errors = 'coerce')
print("Number of observed meteorites in data set:", len(landings))
landings.head()

You may notice that the 'year' column also contains months and days in our dataframe. However, you'll also notice that the date here is always January first. This is not the result of some great cosmic coincidence, the month and day linger simply as an artifact of the data set from the github site itself, and in fact the month and day in this data do not exist. As such, we will be ignoring those quantities in our later analysis. This is a perfect example of data that could be misleading if you're not paying attention! The month and date were not included maliciously - this is more than likely an artifact of date time conversions of the original data set. However, with the appearance of month and days, this data could be unintentionally misleading. 

# Creating a Map

As we have the latitude and longitude of each meteor, a natural first analysis goal is to create a map which will allow us to observe the locations where meteorites have fallen or been discovered. In order to do that, we require `geopandas`, or "Geo-spacial pandas". Geopandas behaves nearly identical to the pandas tool we have used before, however it contains further functionality that makes creating maps significantly more straight forward. In which case, our first task is creating a geopandas data frame, which is done in the code cell below

In [ ]:
# Here we're creating a function which defines our points that we will plot on a map
def create_point(row):
    '''
    Here 'row' is the row of a dataframe that we will use the apply() with this 
    function on. As well, reclong and reclat are the latitude and longitude from our landings 
    data frame. Finally, Point is the Point function that we imported in the beginning of the notebook
    So, all this functio does is return a Point object of latitude and longitude. Easy!
    
    '''
    return Point(row.reclong, row.reclat)

# Now, we create our points by using the apply function on our landings data frame 
points = landings.apply(create_point, axis=1)

print("Below is what our create_point function creates")
print()
print(points.head())

# We are now creating a geopandas data frame with a 'geometry' column as defined 
# by the points we just created. Besides that it is identical to our 'landings'
# data frame 

geo_landings = gpd.GeoDataFrame(landings, geometry=points)

# We also need to define the map projection we're using. In this case, epsg:4326 is the most 
# common projection for a rectangular map. 

geo_landings.crs = {'init': 'epsg:4326'}

# View the first rows, note our 'geometry' tab 
geo_landings.head()

Now that we've created a geopandas data frame, let's take a look at plotting that on a map! First, let's create a simple world map using geopandas, as done below

In [ ]:
# This creates a world map for us! This data set is a part of geopands and included
# in the hub. 
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Now we're just creating a plot as we've done using regular pandas data frames

ax = world.plot(figsize=(20,10), linewidth=0.25, edgecolor='white', color='lightgrey')

Now that we have a basic map, let's put our meteorite landings onto it

In [ ]:

ax = world.plot(figsize=(16,10), linewidth=0.25, edgecolor='white', color='lightgrey')

# Because we have a 'geometry' column, geopandas will plot our meteorite landings without 
# us needing to specify which column to plot. However, we do need to specify that it should appear on the same 
# axis as 
geo_landings.plot(ax=ax, column='fall', alpha = 0.5, markersize=10, legend=True)
ax.axis('off')
plt.show()


Of course, there's around 45000 observed meteorites on the plot above, and that gets messy pretty quickly. In some regard, it might be more interesting to view the meteorite falls and finds for each year. The easiest way to do this in a Jupyter notebook is by using interactive widgets, some of the libraries we imported earlier. However, before we start getting into creating the widget, let's walk through how we would make a map of any year we might be interested first. The first order of buisiness: how do we extract just the year from a date time column?

In [ ]:
'''
To extract the year from a datetime object, we have to be aware of two methods:

dt   : This stands for "datetime" and is used to cast the text in our data frame into a datetime object

year : This extracts the year from the date time object

'''
# This is how we get just the year out!
print("Syntax 1 output:")
print( geo_landings["year"].dt.year.head() )
print()

# Note that this is equivalent, however the first ".year" is in reference to the column name
# and not a method of the date time object.
print("Syntax 2 output:")
print( geo_landings.year.dt.year.head() )

# If our 'year' column was called 'time', this would look like
# geo_landings.time.dt.year.head()


In [ ]:

ax = world.plot(figsize=(16,10), linewidth=0.25, edgecolor='white', color='lightgrey')

# Notice how we're specifying only a single year in this line! 
plot_single_year = geo_landings[geo_landings.year.dt.year == 2012]

plot_single_year.plot(ax=ax, column='fall', alpha = 0.5, markersize=10, legend=True)
plt.title("Meteor Falls in 2012", size = 20)
ax.axis('off')
plt.show()

Feel free to change the code above to look at any year you like in order to get comfortable with the syntax. However, you may have noticed that there is a peculiar point off the coast of Africa in the Atlantic ocean. Let's take a closer look at what's going on there in the cell below. 

In [ ]:
# Here we're looking at the last 10 entries of our dataframe
plot_single_year.tail(10)

Ah ha! There's the problem. In this case the "Northwest Africa" meteorites don't have valid coordinates and were simply zero filled. This is another case of "poor data" that is not intentionally misleading, but can lead to some strange results if you're not careful. One of the most popular ways to deal with missing data in a lot of cases is to simply replace those missing values with zero, so it's best to be careful. In this case, let's filter those points out so they don't show up in our map.

In [ ]:

# If you want to have multiple conditions, the syntax is the same, but each condition is separated
# by an ampersand (&) for 'and'. Also note that '!=' means "not equal to" 

ax = world.plot(figsize=(16,10), linewidth=0.25, edgecolor='white', color='lightgrey')
# Note the changes in the line below
plot_single_year = geo_landings[(geo_landings.year.dt.year == 2012) & (geo_landings.reclat != 0)]

plot_single_year.plot(ax=ax, column='fall', alpha = 0.5, markersize=10, legend=True)
plt.title("Meteor Falls in 2012", size = 20)
ax.axis('off')
plt.show()

There we go! That took care of that peculiar landing sight (far) off the coast of Africa. Now that we understand how to create maps with just single years of data, let's use that to put it all into a interactive widget so we can view any year we feel like!

## Putting it in a widget

In the cell below we're creating the widget that we can use to view any year we like instead. The code here looks long and intimidating, but it's about 95% comments explaining what's going on. The actual code for the widget is but a pittance at approximately 12 lines of code!

In [ ]:
'''
This function is actually nothing new! Instead of plotting our graph directly, we've hidden it 
in a function to call instead. We've also called it 'update' as it is the function that will update
our plot with new data. In this case, our function takes a single input 'y' for year, this is the input 
that is used to filter our data down to a single year. But, everything in the function below is exactly
the plots we've already used. 
'''
def update(y):
    ax = world.plot(figsize=(20,10), linewidth=0.25, edgecolor='white', color='lightgrey')
    
    # Notice how we're specifying the year exactly as we did before, now we're just using
    # the variable 'y' to select the year instead!
    geo_landings[(geo_landings.year.dt.year == y) & (geo_landings.reclat !=0)].plot(alpha=0.75, 
                                                                                    ax=ax,
                                                                                    column = "fall",
                                                                                    legend = True)
    plt.title("Meteorite Falls in " + str(int(y)), size = 20)
    ax.axis('off')
    plt.show()

'''
Here we're simply finding and sorting all the years which had meteorites fall. Here's a quick overview
of what each funciton is doing below, as there's actually a lot to digest in that short line. 

sorted() : This function sorts data in ascending order (smallest -> middlest -> biggest) 
           If it is not simply numerical data, it will be sorted alphanumerically. 
           
list()   : This function simply turns what is inside parenthesis into a python list

landings.year.dt.year.unique() : We've actually seen this before to extract just the year. However this
                                 time we've also added the .unique() method. This returns a filtered list 
                                 of only the unique years that meteorites were observed to fall or found. 
                                   
[:-1]  : This is a short hand to parse our list. In this case the colon specifies that we're taking 
         elements from the beginning of the list. The -1 like this specifies that we're taking the 
         whole list, with the exception of the final element. This is because the year in the final 
         element has an error, and our widget won't work at that point. 
'''

years_with_fall = sorted(list(landings.year.dt.year.unique()))[:-1]


'''
The contents of this code is explained below.

interact_manual             : This is the function that we imported earlier that allows us to create an interactive
                              widget taht will help us explore the data. The _manual suffix tells us that we also
                              want to create a button to only update our plot when we've found the year we desire. 
                  
update                      : This is the function we defined at the beginning of the cell, and the function that 
                              interact will continuously update for us. 

y = widgets.SelectionSlider : This is the widget that we will be using to pass values to our 'update' function. 
                              Notice how we have used the name 'y', the same argument that our update() parameter
                              takes. In this case, SelectionSlider specifies that we want to use a slider widget
                              in order to select the values of y, the year that we're interested in. The arguments
                              of Selection Slider are explaned below. 
                              
description                 : This is a string (computer science talk for "a bunch of text") that will be the label
                              for our slider.
                            
options                     : This is a list of values our slider can take. In ourcase, it is the list we created 
                              earlier where we know meteors have fallen. 
'''

interact_manual(update, 
         y=widgets.SelectionSlider(description='Select Year', 
                                   options=years_with_fall))

We note that you'll have to click "Run Interact" after adjusting your slider to the year of your choice. However, hopefully we've demonstrated how straight forward it is to create an interactive widget of a data set using Jupyter notebooks. If you want to learn more, the documentation for widgets is available [at this link](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html). We note that this link goes directly to the "basics" tutorial of the documentation, rather than the home page. 





# Conclusion

This notebook demonstrated how to work with open data sets that contain a geo-spacial component and how to crate maps using that data. We also demonstrated how you might add a small slice of interactivity to the procedure in order to see meteorite falls at any given year. Finally we also worked through how one might go about exploring other aspects of that data with a classic approach of creating a histogram. In doing so, we demonstrated how you might go about manipulating that data with both logarithmic transformations, and normalization in order to compare data sets of different sizes. The skills we have demonstrated here are ubiquitous to almost any other open-data related project, and can be applied in other analysis of your own.  

![alt text](https://github.com/callysto/callysto-sample-notebooks/blob/master/notebooks/images/Callysto_Notebook-Banners_Bottom_06.06.18.jpg?raw=true)